In [18]:
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset, load_metric
from utils import get_preprocess_function_train, get_preprocess_function_val_test
from t5_lit_mod import LitT5
from torch.utils.data import DataLoader
from lightning.pytorch import Trainer
from pytorch_lightning import LightningModule

In [19]:
model_checkpoint = "t5-small"

In [3]:
# Load dataset
raw_datasets = load_dataset("esnli")

Found cached dataset esnli (/home/jeroen-ov/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc)


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
tokenizer: T5Tokenizer = T5Tokenizer.from_pretrained(model_checkpoint, model_max_length=512)
LightningModule : model = LitT5(model_name=model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True, label_pad_token_id=-100)

In [5]:
train_preprocess_function = get_preprocess_function_train(tokenizer)
preprocessed_train_dataset = raw_datasets['train'].map(train_preprocess_function, batched=True, remove_columns=raw_datasets['train'].column_names)

  0%|          | 0/550 [00:00<?, ?ba/s]

In [8]:
train_dataloader = DataLoader(preprocessed_train_dataset, shuffle=True, batch_size=16, collate_fn=data_collator)

In [9]:
val_preprocess_function = get_preprocess_function_val_test(tokenizer)
preprocessed_val_dataset = raw_datasets['validation'].map(train_preprocess_function, batched=True, remove_columns=raw_datasets["validation"].column_names)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [13]:
val_dataloader = DataLoader(preprocessed_train_dataset, shuffle=True, batch_size=16, collate_fn=data_collator)

In [11]:
trainer = Trainer()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
trainer.fit(model, train_dataloader, val_dataloader)

TypeError: `model` must be a `LightningModule`, got `LitT5`